This tutorial is taken from:

[This Keras blog post](https://keras.io/examples/nlp/lstm_seq2seq/)

I ran into this error:

`NotImplementedError: Cannot convert a symbolic Tensor (2nd_target:0) to a numpy array`

Which was solved by running:

`conda install numpy=1.19`

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [13]:
batch_size = 64  # Batch size for training.
epochs = 3  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "data/keras_character_level_rnn_data/fra.txt"

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input (English) tokens:", num_encoder_tokens)
print("Number of unique output (French) tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input (English) tokens: 71
Number of unique output (French) tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [4]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [5]:
print("The input data can be thought of as {} sheets of {} rows (timesteps/elements in sequence input sequence) with {} columns (vocab size/one-hot encoded featurization)".format(*encoder_input_data.shape))
print(encoder_input_data.shape)

The input data can be thought of as 10000 sheets of 15 rows (timesteps/elements in sequence input sequence) with 71 columns (vocab size/one-hot encoded featurization)
(10000, 15, 71)


In [6]:
print("num_encoder_tokens: {}".format(num_encoder_tokens))
print("num_decoder_tokens: {}".format(num_decoder_tokens))

num_encoder_tokens: 71
num_decoder_tokens: 92


In [7]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
print("For training, these are the shapes required:")
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

For training, these are the shapes required:
(10000, 15, 71)
(10000, 59, 92)
(10000, 59, 92)


In [14]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Train on 8000 samples, validate on 2000 samples
Epoch 1/3
8000/8000 [==============================] - 58s 7ms/sample - loss: 0.6884 - accuracy: 0.8079 - val_loss: 0.7131 - val_accuracy: 0.7977
Epoch 2/3
8000/8000 [==============================] - 54s 7ms/sample - loss: 0.5795 - accuracy: 0.8327 - val_loss: 0.6405 - val_accuracy: 0.8119
Epoch 3/3
8000/8000 [==============================] - 57s 7ms/sample - loss: 0.5279 - accuracy: 0.8453 - val_loss: 0.5940 - val_accuracy: 0.8291


In [15]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [16]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Araisez mois !

-
Input sentence: Go.
Decoded sentence: Araisez mois !

-
Input sentence: Go.
Decoded sentence: Araisez mois !

-
Input sentence: Hi.
Decoded sentence: C'est cous e momment ?

-
Input sentence: Hi.
Decoded sentence: C'est cous e momment ?

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run!
Decoded sentence: Araite !

-
Input sentence: Run.
Decoded sentence: Araisez mois !

-
Input sentence: Run.
Decoded sentence: Araisez mois !

-
Input sentence: Run.
Decoded sentence: Araisez mois !

-
Input sentence: Run.
Decoded sentence: Araisez mois !

-
Input sentence: Run.
Decoded sentence: Araisez mois !

-
Input sen